In [251]:
from qiskit import Aer, QuantumCircuit, QuantumRegister, transpile, execute, IBMQ
import numpy as np
from qiskit.circuit import Qubit
from qiskit.providers.aer.backends.aerbackend import AerBackend
from qiskit.quantum_info import Operator
from qiskit.visualization import plot_histogram
from scipy import linalg

In [252]:
simulator = Aer.get_backend('qasm_simulator')

In [253]:
def load_function_table(name: str) -> list:
    file = open(name, 'r')
    table = file.read().replace('\n', ' ').split(' ')
    file.close()
    for x in range(len(table)):
        table[x] = int(table[x], 16)
    return table


In [254]:
def fast_mobius_transform(table: list) -> list:
    anf = table.copy()
    for i in range(8):
        bit_flip = 1 << i
        for j in range(1 << 8):
            if (j >> i) & 0x1 != 0:
                anf[j] = anf[j] ^ anf[j ^ bit_flip]
    return anf

def anf(table: list) -> list:
    return fast_mobius_transform(table)

In [255]:
# Full Adder
# Register: 0 - x, 1 - y, 2 - carry, 3 - out
def simple_full_adder() -> QuantumCircuit:
     full_adder = QuantumCircuit(4, name='full_adder')
     full_adder.ccx(0, 1, 3)
     full_adder.cx(0, 1)
     full_adder.ccx(1, 2, 3)
     full_adder.cx(1, 2)
     full_adder.cx(0, 1)
     full_adder.swap(2, 3)
     return full_adder

full_adder_gate = simple_full_adder().to_gate()
full_adder_instruction = simple_full_adder().to_instruction()

In [256]:
def full_adder_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    if length == 1:
        return simple_full_adder()

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')
    carry = QuantumRegister(1, 'carry')

    adder = QuantumCircuit(x, y, out, carry, name=format("add_%s", str(length)))

    for i in range(0, length):
        adder.append(full_adder_instruction, [x[i], y[i], carry[0], out[i]])

    return adder

In [257]:
def xor_circuit(length: int) -> QuantumCircuit:
    if length < 1:
        raise Exception('Register length must be positive number')

    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')
    out = QuantumRegister(length, 'out')

    xor = QuantumCircuit(x, y, out, name=format("xor_%s", str(length)))

    for i in range(0, length):
        xor.cx(x[i], out[i])
        xor.cx(y[i], out[i])

    return xor

In [258]:
def vector_to_indices(v: int) -> list:
    ind = []
    for i in range(8):
        if (v >> i) & 0x1 == 0x1:
            ind.append(i)
    return ind


def s_box_circuit(anf: list, box_name: str) -> QuantumCircuit:
    assert len(anf) == (1 << 8)

    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name="s_box_" + box_name)

    for i in range(1 << 8):
        if anf[i] != 0:
            if i == 0:
                for ind in vector_to_indices(anf[i]):
                    circuit.x(out[ind])
            else:
                for ind in vector_to_indices(anf[i]):
                    circuit.mcx(x[vector_to_indices(i)], out[ind])

    return circuit


In [259]:
def add_f8_circuit() -> QuantumCircuit:
    return xor_circuit(8)

In [260]:
# Reduction polynomial x^8 + x^4 + x^3 + x^2 + 1

# Reduction matrix

Q = np.array([
    [1, 0, 1, 1, 1, 0, 0, 0],
    [0, 1, 0, 1, 1, 1, 0, 0],
    [0, 0, 1, 0, 1, 1, 1, 0],
    [0, 0, 0, 1, 0, 1, 1, 1],
    [1, 0, 1, 1, 0, 0, 1, 1],
    [1, 1, 1, 0, 0, 0, 0, 1],
    [1, 1, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 0]
])

In [261]:
print(Q.transpose())


[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [1 0 1 0 1 1 0 0]
 [1 1 0 1 1 0 0 0]
 [1 1 1 0 0 0 1 0]
 [0 1 1 1 0 0 0 0]
 [0 0 1 1 1 0 0 0]
 [0 0 0 1 1 1 0 0]]


In [262]:
# C-NOT Synthesis

Qt = Q.copy().transpose()

In [263]:
Qt[2] = Qt[2] ^ Qt[0]
Qt[3] = Qt[3] ^ Qt[0]
Qt[4] = Qt[4] ^ Qt[0]

In [264]:
Qt[3] = Qt[3] ^ Qt[1]
Qt[4] = Qt[4] ^ Qt[1]
Qt[5] = Qt[5] ^ Qt[1]

In [265]:
Qt[4] = Qt[4] ^ Qt[2]
Qt[5] = Qt[5] ^ Qt[2]
Qt[6] = Qt[6] ^ Qt[2]

In [266]:
Qt[5] = Qt[5] ^ Qt[3]
Qt[6] = Qt[6] ^ Qt[3]
Qt[7] = Qt[7] ^ Qt[3]

In [267]:
Qt[6] = Qt[6] ^ Qt[4]
Qt[7] = Qt[7] ^ Qt[4]

In [268]:
Qt[7] = Qt[7] ^ Qt[5]

In [269]:
print(Qt)

[[1 0 0 0 1 1 1 0]
 [0 1 0 0 0 1 1 0]
 [0 0 1 0 0 0 1 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [270]:
Qtt = np.array(Qt).transpose()

In [271]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [1 0 0 0 1 0 0 0]
 [1 1 0 0 0 1 0 0]
 [1 1 1 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [272]:
Qtt[4] = Qtt[4] ^ Qtt[0]
Qtt[5] = Qtt[5] ^ Qtt[0]
Qtt[6] = Qtt[6] ^ Qtt[0]

In [273]:
Qtt[5] = Qtt[5] ^ Qtt[1]
Qtt[6] = Qtt[6] ^ Qtt[1]

In [274]:
Qtt[6] = Qtt[6] ^ Qtt[2]


In [275]:
print(Qtt)

[[1 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0]
 [0 0 1 0 0 0 0 0]
 [0 0 0 1 0 0 0 0]
 [0 0 0 0 1 0 0 0]
 [0 0 0 0 0 1 0 0]
 [0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 0]]


In [276]:
def qt_reduction_circuit() -> QuantumCircuit:
    qt = QuantumCircuit(8, name="qt_reduction")

    qt.cx(4, 0)
    qt.cx(5, 0)
    qt.cx(6, 0)
    qt.cx(5, 1)
    qt.cx(6, 1)
    qt.cx(6, 2)

    qt.barrier()

    qt.cx(5, 7)
    qt.cx(4, 7)
    qt.cx(4, 6)
    qt.cx(3, 7)
    qt.cx(3, 6)
    qt.cx(3, 5)
    qt.cx(2, 6)
    qt.cx(2, 5)
    qt.cx(2, 4)
    qt.cx(1, 5)
    qt.cx(1, 4)
    qt.cx(1, 3)
    qt.cx(0, 4)
    qt.cx(0, 3)
    qt.cx(0, 2)

    return qt

In [277]:
def explode(*registers) -> list:
    exploded = []
    for reg in registers:
        if isinstance(reg, list):
            exploded.extend(explode(*reg))
        elif isinstance(reg, QuantumRegister):
            for index in range(reg.size):
                exploded.append(reg[index])
        elif isinstance(reg, Qubit):
            exploded.append(reg)
        else:
            raise Exception("Unexpected quantum storage type")
    return exploded


In [278]:
def mult_f8_circuit() -> QuantumCircuit:
    x = QuantumRegister(8, 'x')
    y = QuantumRegister(8, 'y')
    out = QuantumRegister(8, 'out')

    mult = QuantumCircuit(x, y, out, name='mult_f8')

    for i in range(1, 8):
        for j in range(8 - i):
            mult.ccx(x[i + j], y[7 - j], out[i - 1])

    mult.append(qt_reduction_circuit(), qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            mult.ccx(x[j], y[i - j], out[i])

    return mult

In [279]:
# mult_f8_circuit().draw(output='mpl', filename='./mult_8.png', fold=-1)

In [280]:
def add_f64_circuit() -> QuantumCircuit:
    return xor_circuit(64)

In [281]:
def inplace_add_f64_circuit() -> QuantumCircuit:
    return inplace_xor(64)

In [282]:
def mult_const_f8_circuit(constant: int, name=None) -> QuantumCircuit:
    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    const_vector = []
    for i in range(8):
        const_vector.append((constant >> i) & 0x1)

    if name is None:
        name = hex(constant)

    mult = QuantumCircuit(x, out, name='mult_f8_' + name)

    for i in range(1, 8):
        for j in range(8 - i):
            if const_vector[i + j] == 0x1:
                mult.cx(x[7 - j], out[i - 1])

    mult.append(qt_reduction_circuit(), qargs=explode(out))

    for i in range(8):
        for j in range(i + 1):
            if const_vector[j] == 0x1:
                mult.cx(x[i - j], out[i])

    return mult

In [283]:
reduction_poly = 0b00011101
gf_2_8 = [0x1]

elem = 0x1
for _ in range(1, 1 << 8):
    elem = elem << 1
    if elem >> 8 == 0x1:
        elem = (elem ^ reduction_poly) & 0xFF
    gf_2_8.append(elem)

In [284]:
def inplace_xor(length: int):
    x = QuantumRegister(length, 'x')
    out = QuantumRegister(length, 'out')

    circuit = QuantumCircuit(x, out, name='inplace_xor_' + str(length))

    for i in range(length):
        circuit.cx(x[i], out[i])

    return circuit

In [285]:
def inplace_add_f8():
    x = QuantumRegister(8, 'x')
    out = QuantumRegister(8, 'out')

    circuit = QuantumCircuit(x, out, name='inplace_add_f8')

    for i in range(8):
        circuit.cx(x[i], out[i])

    return circuit

In [286]:
def mult_alpha_f64_circuit() -> QuantumCircuit:
    add_f8 = inplace_add_f8().to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_alpha_f64')

    circuit.append(mult_const_f8_circuit(gf_2_8[2  ], 'b^2'  ), qargs=explode(xs[7], outs[0]))
    circuit.append(mult_const_f8_circuit(gf_2_8[70 ], 'b^70' ), qargs=explode(xs[7], outs[3]))
    circuit.append(mult_const_f8_circuit(gf_2_8[224], 'b^224'), qargs=explode(xs[7], outs[4]))
    circuit.append(add_f8, qargs=explode(outs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(gf_2_8[166], 'b^166'), qargs=explode(xs[7], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i], outs[i + 1]))

    return circuit

In [287]:
def mult_inv_alpha_f64_circuit() -> QuantumCircuit:
    add_f8 = inplace_add_f8().to_instruction()

    xs = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    outs = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]

    registers = []
    registers.extend(xs)
    registers.extend(outs)

    circuit = QuantumCircuit(*registers, name='mult_inv_alpha_f64')

    circuit.append(mult_const_f8_circuit(gf_2_8[68 ], 'b^68' ), qargs=explode(xs[0], outs[2]))
    circuit.append(mult_const_f8_circuit(gf_2_8[222], 'b^222'), qargs=explode(xs[0], outs[3]))
    circuit.append(add_f8, qargs=explode(xs[0], outs[4]))
    circuit.append(mult_const_f8_circuit(gf_2_8[164], 'b^164'), qargs=explode(xs[0], outs[5]))
    circuit.append(mult_const_f8_circuit(gf_2_8[168], 'b^168'), qargs=explode(xs[0], outs[6]))

    for i in range(7):
        circuit.append(add_f8, qargs=explode(xs[i + 1], outs[i]))

    return circuit

In [288]:
def not_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')

    circuit = QuantumCircuit(x, name='not')

    for i in range(length):
        circuit.x(i)

    return circuit

In [289]:
L = [[0x01, 0x01, 0x05, 0x01, 0x08, 0x6, 0x07, 0x04],
    [0x04, 0x01, 0x01, 0x05, 0x01, 0x08, 0x06, 0x07],
    [0x07, 0x04, 0x01, 0x01, 0x05, 0x01, 0x08, 0x06],
    [0x06, 0x07, 0x04, 0x01, 0x01, 0x05, 0x01, 0x08],
    [0x08, 0x06, 0x07, 0x04, 0x01, 0x01, 0x05, 0x01],
    [0x01, 0x08, 0x06, 0x07, 0x04, 0x01, 0x01, 0x05],
    [0x05, 0x01, 0x08, 0x06, 0x07, 0x04, 0x01, 0x01],
    [0x01, 0x05, 0x01, 0x08, 0x06, 0x07, 0x04, 0x01]]

In [290]:
def strumok_substitution_circuit() -> QuantumCircuit:
    reset_8 = reset_circuit(8)

    x = [QuantumRegister(8, 'x_' + str(i)) for i in range(8)]
    out = [QuantumRegister(8, 'out_' + str(i)) for i in range(8)]
    tmp = QuantumRegister(8, 'tmp')

    regs = []
    regs.extend(x)
    regs.extend(out)
    regs.append(tmp)

    circuit = QuantumCircuit(*regs, name='strumok_tau')

    for i in range(8):
        index = i % 4
        s_box = s_box_circuit(anf(load_function_table('./sbox/pi_' + str(index) + '.txt')), str(index))

        circuit.append(reset_8, qargs=explode(tmp))
        circuit.append(s_box, qargs=explode(x[i], tmp))

        for j in range(8):
            circuit.append(mult_const_f8_circuit(L[j][i]), qargs=explode(tmp, out[j]))

    return circuit

In [291]:
def swap_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')
    y = QuantumRegister(length, 'y')

    circuit = QuantumCircuit(x, y, name='swap_' + str(length))

    for i in range(length):
        circuit.swap(x[i], y[i])

    return circuit

In [292]:
def reset_circuit(length: int) -> QuantumCircuit:
    x = QuantumRegister(length, 'x')

    circuit = QuantumCircuit(x, name='reset_' + str(length))

    for i in range(length):
        circuit.reset(x[i])

    return circuit

In [293]:
def strumok_fsm_circuit() -> QuantumCircuit:
    x = QuantumRegister(64, 'x')
    y = QuantumRegister(64, 'y')
    z = QuantumRegister(64, 'z')
    out = QuantumRegister(64, 'out')
    carry = QuantumRegister(1, 'carry')

    circuit = QuantumCircuit(x, y, z, out, carry, name='strumok_fsm')
    circuit.append(full_adder_circuit(64), qargs=explode(x, y, out, carry))
    circuit.append(inplace_xor(64), qargs=explode(z, out))

    return circuit


In [294]:
def strumok_next_circuit(init_mode=False) -> QuantumCircuit:
    swap_64 = swap_circuit(64).to_instruction()
    reset_64 = reset_circuit(64).to_instruction()
    iadd_f64 = inplace_add_f64_circuit().to_instruction()

    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    tmp = QuantumRegister(64, 'tmp')
    carry = QuantumRegister(1, 'carry')

    # Circuit Input
    regs = []
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(tmp)
    regs.append(carry)

    circuit = QuantumCircuit(*regs, name='strumok_next')

    circuit.append(strumok_substitution_circuit(), qargs=explode(r_1, r_2, tmp[0:8]))
    circuit.append(reset_64, qargs=explode(r_1))
    circuit.append(full_adder_circuit(64), qargs=explode(r_2, s[13], r_1, carry[0]))

    circuit.append(reset_64, qargs=explode(tmp))
    circuit.append(iadd_f64, qargs=explode(s[13], tmp))
    circuit.append(mult_alpha_f64_circuit(), qargs=explode(s[0], tmp))
    circuit.append(mult_inv_alpha_f64_circuit(), qargs=explode(s[11], tmp))

    if init_mode:
        circuit.reset(carry[0])
        circuit.append(strumok_fsm_circuit(), qargs=explode(s[15], r_1, r_2, tmp, carry[0]))

    # After this rotation s[i] -> s[i - 1], where s[0] -> s[15]
    for j in range(14, -1, -1):
        circuit.append(swap_64, explode(s[15], s[j]))

    circuit.append(reset_64, qargs=explode(s[15]))
    circuit.append(iadd_f64, qargs=explode(tmp, s[15]))

    return circuit

In [295]:
# STRUMOK-256
def strumok_init_circuit() -> QuantumCircuit:
    not_64 = not_circuit(64).to_instruction()
    cnot_64 = inplace_xor(64).to_instruction()
    reset_64 = reset_circuit(64).to_instruction()

    strumok_next = strumok_next_circuit(True).to_instruction()

    # Cipher Input
    key = [QuantumRegister(64, 'k_' + str(i)) for i in range(4)]
    iv = [QuantumRegister(64, 'iv_' + str(i)) for i in range(4)]

    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    tmp = QuantumRegister(64, 'tmp')
    carry = QuantumRegister(1, 'carry')

    # Circuit Input
    regs = []
    regs.extend(key)
    regs.extend(iv)
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(tmp)
    regs.append(carry)

    circuit = QuantumCircuit(*regs, name='strumok_init')

    circuit.append(cnot_64, qargs=explode(key[0], s[15]))
    circuit.append(not_64, qargs=explode(s[15]))
    circuit.append(cnot_64, qargs=explode(key[1], s[14]))
    circuit.append(cnot_64, qargs=explode(key[2], s[13]))
    circuit.append(not_64, qargs=explode(s[13]))
    circuit.append(cnot_64, qargs=explode(key[3], s[12]))
    circuit.append(cnot_64, qargs=explode(key[0], s[11]))
    circuit.append(cnot_64, qargs=explode(key[1], s[10]))
    circuit.append(not_64, qargs=explode(s[10]))
    circuit.append(cnot_64, qargs=explode(key[2], s[9]))
    circuit.append(cnot_64, qargs=explode(key[3], s[8]))
    circuit.append(cnot_64, qargs=explode(key[0], s[7]))
    circuit.append(not_64, qargs=explode(s[7]))
    circuit.append(cnot_64, qargs=explode(key[1], s[6]))
    circuit.append(not_64, qargs=explode(s[6]))
    circuit.append(cnot_64, qargs=explode(key[2], s[5]))
    circuit.append(cnot_64, qargs=explode(iv[3], s[5]))
    circuit.append(cnot_64, qargs=explode(key[3], s[4]))
    circuit.append(cnot_64, qargs=explode(key[0], s[3]))
    circuit.append(cnot_64, qargs=explode(iv[2], s[3]))
    circuit.append(cnot_64, qargs=explode(key[1], s[2]))
    circuit.append(cnot_64, qargs=explode(iv[1], s[2]))
    circuit.append(cnot_64, qargs=explode(key[2], s[1]))
    circuit.append(cnot_64, qargs=explode(key[3], s[0]))
    circuit.append(cnot_64, qargs=explode(iv[3], s[0]))

    for _ in range(32):
        circuit.append(strumok_next, qargs=explode(s, r_1, r_2, tmp, carry))
        circuit.reset(carry[0])
        circuit.append(reset_64, qargs=explode(tmp))

    return circuit


In [296]:
def strumok_stream_circuit() -> QuantumCircuit:
    # Linear Feedback Shift Register State
    s = [QuantumRegister(64, 's_' + str(i)) for i in range(16)]

    # Finite State Machine
    r_1 = QuantumRegister(64, 'r_1')
    r_2 = QuantumRegister(64, 'r_2')

    # Ancillary State
    carry = QuantumRegister(1, 'carry')

    # Stream output
    out = QuantumRegister(64, 'out')

    # Circuit Input
    regs = []
    regs.extend(s)
    regs.append(r_1)
    regs.append(r_2)
    regs.append(carry)
    regs.append(out)

    circuit = QuantumCircuit(*regs, name='strumok_next')
    circuit.append(inplace_add_f64_circuit(), qargs=explode(s[0], out))
    circuit.append(strumok_fsm_circuit(), qargs=explode(s[15], r_1, r_2, out, carry))

    return circuit
